<a href="https://colab.research.google.com/github/19990715/BART-/blob/main/subtract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets

In [ ]:
! pip install transformers

In [ ]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l
!pip install -U mxnet-cu101==1.7.0

In [ ]:
#!pip install rouge/requirements.txt
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rouge-metrix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement rouge-metrix (from versions: none)
ERROR: No matching distribution found for rouge-metrix


In [ ]:
!pip3 install torchvision

In [ ]:
import torch
x = torch.Tensor(5, 3)
print(x)

tensor([[2.0315e-35, 0.0000e+00, 3.7835e-44],
        [0.0000e+00,        nan, 0.0000e+00],
        [1.3733e-14, 6.4069e+02, 4.3066e+21],
        [1.1824e+22, 4.3066e+21, 6.3828e+28],
        [3.8016e-39, 3.8016e-39, 0.0000e+00]])


In [ ]:
#prepare the dataset
#trainscale和里面的参数
from datasets import Dataset

class Data:
    def __init__(self, data_path, tokenizer):
        self.path = data_path
        self.max_input_length = 1024
        self.max_target_length = 150
        # self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
        self.tokenizer = tokenizer
        
    def preprocess(self, train_scale=0.6):
        with open(self.path,'r') as f:
            raw_data = f.readlines()
        print(f"=======data_len: {len(raw_data)}")
        start = int(len(raw_data)*train_scale)
        print(f"======train_len: {start}")
        
        raw_train_data = raw_data[:start]
        raw_test_data = raw_data[start:]
        raw_train_test_data = {'train':{'id':[],'document':[],'summary':[]}, \
                               'test':{'id':[],'document':[],'summary':[]}}        
        
        for i,item in enumerate(raw_train_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['train']['id'].append(i)
            
            # document is the train data, summary is the substract label
            raw_train_test_data['train']['summary'].append(label.strip())
            raw_train_test_data['train']['document'].append(text.strip())

        for j,item in enumerate(raw_test_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['test']['id'].append(i+j+1)
            raw_train_test_data['test']['summary'].append(label.strip())
            raw_train_test_data['test']['document'].append(text.strip())
        
        def preprocess_function(examples):
            
            inputs = examples['document']
            model_inputs = self.tokenizer(inputs, max_length = self.max_input_length, padding = 'max_length', truncation=True)
            
            with self.tokenizer.as_target_tokenizer():
                labels = self.tokenizer(examples['summary'], max_length = self.max_target_length, padding = 'max_length', truncation = True)
            model_inputs['labels'] = labels['input_ids']
            return model_inputs
        
        train_dataset = Dataset.from_dict(raw_train_test_data['train'])
        test_dataset = Dataset.from_dict(raw_train_test_data['test'])
        tokenized_train_dataset = train_dataset.map(preprocess_function)
        tokenized_test_dataset = test_dataset.map(preprocess_function)
        return tokenized_train_dataset, tokenized_test_dataset


In [ ]:
def preprocess(path, train_scale=0.6):
        with open(path,'r') as f:
            raw_data = f.readlines()
        print(f"=======data_len: {len(raw_data)}")
        start = int(len(raw_data)*train_scale)
        print(f"======train_len: {start}")
        
        raw_train_data = raw_data[:start]
        raw_test_data = raw_data[start:]
        raw_train_test_data = {'train':{'id':[],'document':[],'summary':[]}, \
                               'test':{'id':[],'document':[],'summary':[]}} 
        #print(raw_train_data)
       
        for i,item in enumerate(raw_train_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['train']['id'].append(i)
            
            # document 是训练数据, summary是label
            raw_train_test_data['train']['summary'].append(label.strip())
            raw_train_test_data['train']['document'].append(text.strip())

        for j,item in enumerate(raw_test_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['test']['id'].append(i+j+1)
            raw_train_test_data['test']['summary'].append(label.strip())
            raw_train_test_data['test']['document'].append(text.strip())

        print(raw_train_test_data['train']['summary'])

preprocess("substract_data.txt")

=======data_len: 38
======train_len: 30
['Stocks were mixed this week,investors weighed high from technology companies.', 'Numerous economic reports need to be digest, including durable goods reports and unemployment rate.', 'Sustainable Impact Investing gained traction.']


In [ ]:
#load the model
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartForConditionalGeneration
from transformers import AutoTokenizer

#checkpoint = "distilbart-xsum-9-6"
checkpoint = "sshleifer/distilbart-xsum-9-6"
model = BartForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)



Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
#evaluating metric use the rouge_score
import numpy as np
from rouge_score import rouge_scorer, scoring
import nltk


def compute(predictions, references, rouge_types=None, use_agregator=True, use_stemmer=False):
    if rouge_types is None:
        rouge_types = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

    scorer = rouge_scorer.RougeScorer(rouge_types=rouge_types, use_stemmer=use_stemmer)
    if use_agregator:
        aggregator = scoring.BootstrapAggregator()
    else:
        scores = []

    for ref, pred in zip(references, predictions):
        score = scorer.score(ref, pred)
        if use_agregator:
            aggregator.add_scores(score)
        else:
            scores.append(score)

    if use_agregator:
        result = aggregator.aggregate()
    else:
        result = {}
        for key in scores[0]:
            result[key] = list(score[key] for score in scores)

    return result


#metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    print("1")

    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
#evaluating metric use the rouge_score
import numpy as np
from rouge_score import rouge_scorer, scoring
import nltk


def compute(predictions, references, rouge_types=None, use_agregator=True, use_stemmer=False):
    if rouge_types is None:
        rouge_types = ["rouge1", "rougeL"]

    scorer = rouge_scorer.RougeScorer(rouge_types=rouge_types, use_stemmer=True)
    if use_agregator:
        aggregator = scoring.BootstrapAggregator()
    else:
        scores = []

    for ref, pred in zip(references, predictions):
        score = scorer.score(ref, pred)
        if use_agregator:
            aggregator.add_scores(score)
        else:
            scores.append(score)

    if use_agregator:
        result = aggregator.aggregate()
    else:
        result = {}
        for key in scores[0]:
            result[key] = list(score[key] for score in scores)

    return result


#metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    #prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    #result["gen_len"] = np.mean(prediction_lens)
    
    print("1")

    return {k: round(v, 4) for k, v in result.items()}



In [ ]:
predictions, labels = eval_pred
print(compute_metrics(eval_pred))

NameError: ignored

In [ ]:
#Configure Hyperparameters
batch_size = 1
args = Seq2SeqTrainingArguments( \
    "/Users/fusi/Desktop/ds/individual_project/demo/model", \
    evaluation_strategy = 'steps', \
    learning_rate = 5e-5, \
    per_device_train_batch_size = batch_size, \
    per_device_eval_batch_size = batch_size, \
    weight_decay = 0.2, \
    save_steps = 5, \
    #save_total_limit = 5, \
    num_train_epochs = 3, \
    predict_with_generate = True, \
    eval_steps = 5, \
    logging_first_step=True, \
    report_to="none")


PyTorch: setting up devices


In [ ]:
#use transformers api to train the data
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, padding=True)
data = Data('substract_data.txt', tokenizer)
tokenized_train_dataset, tokenized_test_dataset = data.preprocess()
trainer =Seq2SeqTrainer( \
                       model, \
                       args, \
                       train_dataset = tokenized_train_dataset, \
                       eval_dataset = tokenized_test_dataset, \
                       data_collator = data_collator, \
                       tokenizer = tokenizer, \
                       compute_metrics = compute_metrics)



=======data_len: 48
======train_len: 28


  0%|          | 0/3 [00:00<?, ?ex/s]

In [ ]:
#use transformers api to train the data
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, padding=True)
data = Data('substract_data.txt', tokenizer)
tokenized_train_dataset, tokenized_test_dataset = data.preprocess()

trainer =Seq2SeqTrainer( \
                       model, \
                       args, \
                       train_dataset = tokenized_train_dataset, \
                       eval_dataset = tokenized_test_dataset, \
                       data_collator = data_collator, \
                       tokenizer = tokenizer, \
                       compute_metrics = compute_metrics)

=======data_len: 48
======train_len: 28


  0%|          | 0/3 [00:00<?, ?ex/s]

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document. If summary, id, document are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 9


Step,Training Loss,Validation Loss
5,2.818800,No log


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document. If summary, id, document are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 0
  Batch size = 1
Saving model checkpoint to /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5
Configuration saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/config.json
Model weights saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/pytorch_model.bin
tokenizer config file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/tokenizer_config.json
Special tokens file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9, training_loss=1.4867907100253634, metrics={'train_runtime': 200.1368, 'train_samples_per_second': 0.045, 'train_steps_per_second': 0.045, 'total_flos': 11841646362624.0, 'train_loss': 1.4867907100253634, 'epoch': 3.0})

In [ ]:
train_result = trainer.train() 
train_metrics = train_result.metrics
evaluate_metrics = trainer.evaluate()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document. If summary, id, document are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 15


Step,Training Loss,Validation Loss
5,2.833700,No log
10,2.833700,No log
15,2.833700,No log


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document. If summary, id, document are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 0
  Batch size = 1
Saving model checkpoint to /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5
Configuration saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/config.json
Model weights saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/pytorch_model.bin
tokenizer config file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/tokenizer_config.json
Special tokens file saved in /Users/fusi/Desktop/ds/individual_project/demo/model/checkpoint-5/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGe

In [ ]:
print(train_metrics)
print(evaluate_metrics)

{'train_runtime': 339.4548, 'train_samples_per_second': 0.044, 'train_steps_per_second': 0.044, 'total_flos': 19736077271040.0, 'train_loss': 2.803520123163859, 'epoch': 5.0}
{'eval_runtime': 0.0087, 'eval_samples_per_second': 0.0, 'eval_steps_per_second': 0.0, 'epoch': 5.0}


In [ ]:
#Configure Hyperparameters
batch_size = 1
args = Seq2SeqTrainingArguments( \
    "/Users/fusi/Desktop/ds/individual_project/demo/model", \
    #Evaluation is done (and logged) every eval_steps
    evaluation_strategy = 'steps',\
    #The initial learning rate for Adam （5e-5） 
    learning_rate = 3e-5, \ 
    #The batch size per GPU/TPU core/CPU for training（8）
    per_device_train_batch_size = batch_size, \
    #The batch size per GPU/TPU core/CPU for evaluation（8）
    per_device_eval_batch_size = batch_size, \
    #The weight decay to apply (if not zero)
    weight_decay = 0.1, \
    #Number of updates steps before two checkpoint saves（500）
    save_steps = 100, \
    #If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir
    save_total_limit = 10, \
    #要执行的训练 epoch 总数（3.0）
    num_train_epochs = 5, \
    #是否使用 generate 来计算生成度量（ROUGE、BLEU）
    predict_with_generate = True, \
    #是否使用 16 位（混合）精度训练（通过 NVIDIA Apex）而不是 32 位训练（默认false）
    fp16 = True, \
    #Number of update steps between two evaluations
    eval_steps = 200, \
    #是否记录和评估第一个global_step
    logging_first_step=True)


SyntaxError: ignored

In [ ]:
#maxlength和minlength要调一下

def predict(sentence):
    inputs = tokenizer([sentence],max_length = 1024, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], num_beams=70, max_length=150,min_length=20,early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return ' '.join(summary)


In [ ]:
print(predict("Stocks scored a win in the trading week ended March 27-- as in the best market week in 82 years, even as Friday of that week was a down day. The rally had some investors wondering if 'the low was in.' But in past big selloffs, the confirming low re-test has come months later."))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Stocks were trading at record highs for the first time in more than 30 years.


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
print(scores)

{'rouge1': Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765), 'rougeL': Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)}


In [ ]:
l = []
pre = [] 
with open('summary.txt','r') as f :
  for line in f.readlines():
    line = line.strip('\n')
    l.append(line)
print(l)

['Stocks rose on Friday, finishing off a strong week that saw major indexes climb higher. Riskier equities outperformed blue chips, a trend that was commonplace during the early weeks of the recovery. On Friday, the Dow Jones Industrial Average gained 162 points, or 0.6%. The S&P 500 was up 0.7% and the Nasdaq Composite was up 0.6%. For the full week, the Dow was up 2.6%, the S&P 500 was up 3.3% and the Nasdaq gained 3.4%. Year-to-date, the DJIA crossed into positive territory this week, and is up 0.4%, the S&P is up 8.6%, and the Nasdaq is up 30.4%. Over the past 52 weeks, the Dow is up 8.5%, the S&P 500 is up 19.9%, and the Nasdaq is up 46.9%. ', 'Apple manufactures Smartphones, tablets, PCs, software and peripherals for a worldwide customer base. Its products include Mac desktop and mobile PCs, iPhone, iPad, Apple Watch, and various consumer products, including AirPods, Beats headphones and Apple TV. Apple services include App Store, iTunes, iCloud, Apple TV+, Apple Arcade, Apple Mu

In [ ]:
#print(predict(l[0]))
for i in l:
  p = predict(i)
  pre.append(p)

In [ ]:
#print(pre)
import pandas as pd
df = pd.DataFrame()
df['summary'] = l
df['subtract'] = pre
df

NameError: ignored